In [16]:
# Select CUDA device index
import os
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration, pipeline
from datasets import load_dataset

import torchaudio
import torchaudio.transforms as T

#model_name_or_path = "openai/whisper-large-v3"
model_name_or_path = "distil-whisper/distil-large-v3"
language = "id"



device = "cuda:1" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model = WhisperForConditionalGeneration.from_pretrained(
    model_name_or_path, torch_dtype=torch.float32, low_cpu_mem_usage=True, use_safetensors=True,
)
model.to(device)

processor = WhisperProcessor.from_pretrained(model_name_or_path)


def transcribe(procesor, model, file, language='en'):
    audio, rate = torchaudio.load(file)

    if rate != 16000:
        resampler = T.Resample(orig_freq=rate, new_freq=16000)
        audio = resampler(audio)
        rate = 16000

    input_features = processor(audio.squeeze(), sampling_rate=rate, return_tensors="pt").input_features.to(device)
    forced_decoder_ids = processor.tokenizer.get_decoder_prompt_ids(language=language, task="transcribe")
    generated_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return transcription


In [17]:
def file_list(folder_path):
    try:
        files = os.listdir(folder_path)
        return [os.path.abspath(os.path.join(folder_path, file)) for file in files if os.path.isfile(os.path.join(folder_path, file))]
    except FileNotFoundError:
        return f"Error: {folder_path} not found"

In [18]:
#fl = file_list("../indonesian-english")
fl = file_list("../cambodian-english")
hyps = []
for r in fl:
    hyps.append(transcribe(processor, model, r, language='en'))
print(hyps)

You have passed task=transcribe, but also have set `forced_decoder_ids` to [(1, 50259), (2, 50360), (3, 50364)] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.


[' Please call Stella, ask her to bring these things with her from the store, 6-spon, fresno pe, 5-takes slab of blue cheese, and a maybe snack for her brother Bob. We also need a small plastic snake and a big toy frog for the kid. She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.', ' Please call Stella, ask her to bring this thing with her from the store, six spoon of the fresh snow-piece, five thick slab of the blue cheese, and maybe snack for her brother Bob. We also need small plastic snake and the big toy fork for the kid. She can scoop this thing into three red bag and we will go meet her Wednesday at the train station.', ' I have a lunch day unit that has never given me any trouble.', ' please call stella ask her to bring these things with her from the store six spoons of fresh snow-piece five thick-slaps of blue cheese and maybe snack for her brother bob we also need a small plastic snake and a big toy frog for the kids she 

In [19]:
# Indonesian
# ref_str = """Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# I got some snacks, a few beers and I get ready to watch.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# I have launched their unit that had never given me any trouble.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# Farm percent
# beautiful-myself
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# greenhouse discover
# Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
# the-doctor-said-he-had-already-been-a-lot-better-now"""

# Cambodian
ref_str = """Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
Please call Stella.  Ask her to bring these things with her from the store:  Six spoons of fresh snow peas, five thick slabs of blue cheese, and maybe a snack for her brother Bob.  We also need a small plastic snake and a big toy frog for the kids.  She can scoop these things into three red bags, and we will go meet her Wednesday at the train station."""


refs = ref_str.splitlines()

In [20]:
import jiwer

wer = jiwer.wer(refs, hyps)
print(f'WER = {wer*100}')
cer = jiwer.cer(refs, hyps)
print(f'CER = {cer*100}')

WER = 20.88607594936709
CER = 9.31480126798342


In [21]:
for i in range(len(refs)):
    #print(f"{i+1}: {refs[i]}\t{hyps[i]}")
    print(f"{hyps[i]}")

 Please call Stella, ask her to bring these things with her from the store, 6-spon, fresno pe, 5-takes slab of blue cheese, and a maybe snack for her brother Bob. We also need a small plastic snake and a big toy frog for the kid. She can scoop these things into three red bags, and we will go meet her Wednesday at the train station.
 Please call Stella, ask her to bring this thing with her from the store, six spoon of the fresh snow-piece, five thick slab of the blue cheese, and maybe snack for her brother Bob. We also need small plastic snake and the big toy fork for the kid. She can scoop this thing into three red bag and we will go meet her Wednesday at the train station.
 I have a lunch day unit that has never given me any trouble.
 please call stella ask her to bring these things with her from the store six spoons of fresh snow-piece five thick-slaps of blue cheese and maybe snack for her brother bob we also need a small plastic snake and a big toy frog for the kids she can scoop t